In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StringType, StructType, StructField, TimestampType, IntegerType
from pyspark.sql.functions import from_json, col

# Define the path to the jars on the EC2 instance
spark_jars_path = "/home/ec2-user/stream-processing-template/jars"  # <-- Update this path

spark = SparkSession.builder.appName("retail_pysaprk_consumer") \
    .config("spark.jars", f"{spark_jars_path}/commons-pool2-2.11.1.jar,"
            f"{spark_jars_path}/spark-sql-kafka-0-10_2.12-3.4.0.jar,"
            f"{spark_jars_path}/spark-streaming-kafka-0-10-assembly_2.12-3.4.0.jar") \
    .getOrCreate()


23/12/12 16:58:03 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
/usr/local/lib/python3.7/site-packages/pyspark/context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


In [2]:
# Define the schema for our data
schema = StructType([
    StructField("store_location", StringType(), True),
    StructField("time_of_purchase", TimestampType(), True),
    StructField("product_ID", StringType(), True),
    StructField("transaction_amount", IntegerType(), True)
])

# Stream from Kafka topic
df = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", "b-1.monstercluster1.6xql65.c3.kafka.eu-west-2.amazonaws.com:9092") \
    .option("subscribe", "retail_transactions") \
    .load()

In [3]:
import time
from pyspark.sql.functions import from_json, col, sum

transactions = (df.selectExpr("CAST(value AS STRING)")
                .withColumn("data", from_json(col("value"), schema))
                .select("data.*"))

total_by_location = transactions.groupBy("store_location") \
    .agg(sum("transaction_amount").alias("total_transaction_amount"))


query = transactions.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# query.awaitTermination()
# query.awaitTermination(timeout=3000)  # Wait for a maximum of 60,000 milliseconds (3 seconds)

# Wait for 3 seconds
time.sleep(5)

# Stop the streaming query
query.stop()

23/12/12 16:58:28 WARN ResolveWriteToStream: Temporary checkpoint location created which is deleted normally when the query didn't fail: /tmp/temporary-98376728-b5b8-49e6-9388-8ce3edaf291b. If it's required to delete it under any circumstances, please set spark.sql.streaming.forceDeleteTempCheckpointLocation to true. Important to know deleting temp checkpoint folder is best effort.
23/12/12 16:58:28 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/12/12 16:58:30 WARN AdminClientConfig: These configurations '[key.deserializer, value.deserializer, enable.auto.commit, max.poll.records, auto.offset.reset]' were supplied but are not used yet.


-------------------------------------------
Batch: 0
-------------------------------------------
+--------------+----------------+----------+------------------+
|store_location|time_of_purchase|product_ID|transaction_amount|
+--------------+----------------+----------+------------------+
+--------------+----------------+----------+------------------+



-------------------------------------------
Batch: 1
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|      New York|2023-12-12 16:58:32|    P00034|              -411|
|       Houston|2023-12-12 16:58:34|    P00003|               480|
+--------------+-------------------+----------+------------------+

-------------------------------------------
Batch: 2
-------------------------------------------
+--------------+-------------------+----------+------------------+
|store_location|   time_of_purchase|product_ID|transaction_amount|
+--------------+-------------------+----------+------------------+
|   Los Angeles|2023-12-12 16:58:36|    P00009|               331|
+--------------+-------------------+----------+------------------+



In [ ]:
import time
from pyspark.sql.functions import from_json, col, sum

transactions = (df.selectExpr("CAST(value AS STRING)")
                .withColumn("data", from_json(col("value"), schema))
                .select("data.*"))

total_by_location = transactions.groupBy("store_location") \
    .agg(sum("transaction_amount").alias("total_transaction_amount"))


query = transactions.writeStream \
    .outputMode("append") \
    .format("console") \
    .start()

# query.awaitTermination()
# query.awaitTermination(timeout=3000)  # Wait for a maximum of 60,000 milliseconds (3 seconds)

# Wait for 3 seconds
time.sleep(10)

total_by_location.show()

# Stop the streaming query
query.stop()